In [ ]:
import gym
import numpy as np
import random
from ray.rllib.policy.policy import Policy
from gym import spaces
import mpu

In [ ]:
class PlaystyleHeuristicPolicy(Policy):

    def __init__(self, Playstyle, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.Playstyle = Playstyle
        self.HandScores = mpu.io.read('Heurisitc_Val_Tables/hand_value_table_norm_int8_24_100.pickle')
        # could make a self round history of actions

    def get_initial_state(self):
        return self.Playstyle  # check the uses of this func 

    def compute_actions(
        self,
        obs_batch,
        state_batches=None,
        prev_action_batch=None,
        prev_reward_batch=None,
        info_batch=None,
        episodes=None,
        **kwargs
    ):
        obs = obs_batch[0][0:24] + obs_batch[0][24:]            # observes its own hands out of the possible 24 against the other 22??
        obs = np.array(np.where(obs==1)[0], dtype=np.int8)      # Formating the cards into array of type int
        value = self.HandScores[obs.tobytes()]                  # Uses lookup table to map the hand values to the hands in your possession
        # rand = np.random.random()                               # Send An action that is based in randomness to maintain heuristic nature

        if self.Playstyle == 0:  # Aggressive
            # if the value of card is > X the raise in the preflop.
            # When is a Preflop state
            # what is output of 'value' and what is a good set threshold
            # needs to monitor its own preflop betting rounds history to calculate a PFR score. % Hands raised vs played
            if value > X:
                return [2], [], {} # Raise
            elif value > Y:
                return [1], [], {} #Call
            else:
                return [0], [], {} #Fold
        else: # Passive
            # Large gap between VPIP (Voluntarily Put in Pot) and PRF (Pre Flop Raise)
            # take calculated PFR from last step
            # VPIP needs to know:
                # History of Rounds because:
                    # Hands played
                    # % of those hands where you voluntarily put a chip into the pot in the pre flop. i.e. anything other than a forced blind input.
                        # Action taken per historic round
            # if VPIP - PRF
            if value > X:
                return [2], [], {} # Raise
            elif value > Y:
                return [1], [], {} #Call
            else:
                return [0], [], {} #Fold

    def get_weights(self):
        return None

    def set_weights(self, weights):
        return None